In [79]:
import pandas as pd
import numpy as np
import math
from underscore import _
import os
import sys
import TreeNode

ModuleNotFoundError: No module named 'TreeNode'

In [ ]:
class DecisionTree:
    
    
    def preprocessData(self,filename):
        data=pd.read_csv(filename, sep=" ", header=None)
        data.columns=['index','occupied','prices','music','location','VIP','favorite beer','Enjoy']
        data=data[1:]
        data=data.drop('index',axis=1)
        data['occupied']=data['occupied'].str.replace(',','')
        data['prices']=data['prices'].str.replace(',','')
        data['music']=data['music'].str.replace(',','')
        data['location']=data['location'].str.replace(',','')
        data['VIP']=data['VIP'].str.replace(',','')
        data['favorite beer']=data['favorite beer'].str.replace(',','')
        data['Enjoy']=data['Enjoy'].str.replace(';','')
        #data=data.sample(frac=1).reset_index(drop=True)
        training_data=data[0:17]
        attributes=list(data.columns)
        target=attributes[-1]
        training_data=training_data.values.tolist()
        testingData=data.ix[17:21,0:6]
        test_labels=data.ix[17:21,6]
        testingData=testingData.values.tolist()
        return training_data, testingData, attributes, target, test_labels, data
    
    def entropy(self,attributes, data, targetAttr):
        valFreq = {}
        dataEntropy = 0.0
        #find index of the target attribute
        i = 0
        i=attributes.index(targetAttr)
    
        # Calculate the frequency of each of the values in the target attr
        for entry in data:
            if entry[i] in valFreq:
                valFreq[entry[i]] += 1.0
            else:
                valFreq[entry[i]]  = 1.0

        # Calculate the entropy of the data for the target attr
        for freq in valFreq.values():
            dataEntropy += (-freq/len(data)) * math.log(freq/len(data), 2) 
        
        return dataEntropy
    
    def gain(self,attributes, data, attr, targetAttr):
        """
        Calculates the information gain (reduction in entropy) that would
        result by splitting the data on the chosen attribute (attr).
        """
        valFreq = {}
        subsetEntropy = 0.0
    
        #find index of the attribute
        i = attributes.index(attr)

        # Calculate the frequency of each of the values in the target attribute
        for entry in data:
            if entry[i] in valFreq:
                valFreq[entry[i]] += 1.0
            else:
                valFreq[entry[i]]  = 1.0
        # Calculate the sum of the entropy for each subset of records weighted
        # by their probability of occuring in the training set.
        for val in valFreq.keys():
            valProb        = valFreq[val] / sum(valFreq.values())
            dataSubset     = [entry for entry in data if entry[i] == val]
            subsetEntropy += valProb * self.entropy(attributes, dataSubset, targetAttr)

        # Subtract the entropy of the chosen attribute from the entropy of the
        # whole data set with respect to the target attribute (and return it)
        return (self.entropy(attributes, data, targetAttr) - subsetEntropy)
    
    def chooseAttr(self,data, attributes, target):
        best = attributes[0]
        maxGain = 0;
        for attr in attributes:
            if attr!='Enjoy':
                newGain = self.gain(attributes, data, attr, target) 
                if newGain>maxGain:
                    maxGain = newGain
                    best = attr
        return best

    #get values in the column of the given attribute 
    def getValues(self,data, attributes, attr):
        index = attributes.index(attr)
        values = []
        for entry in data:
            if entry[index] not in values:
                values.append(entry[index])
        return values

    def getExamples(self,data, attributes, best, val):
        examples = [[]]
        index = attributes.index(best)
        for entry in data:
            #find entries with the give value
            if (entry[index] == val):
                newEntry = []
                #add value if it is not in best column
                for i in range(0,len(entry)):
                    if(i != index):
                        newEntry.append(entry[i])
                examples.append(newEntry)
        examples.remove([])
        return examples

    def majority(self,data,attributes,target):
        valFreq={}
        index=attributes.index(target)
        priority=['occupied','prices','music','location'
                 ,'VIP','favorite beer','Enjoy']
        for v in data:
            if v[index] in valFreq:
                valFreq[v[index]]+=1
            else:
                valFreq[v[index]]=1
        maxAttr=0
        major="" 
        for key in valFreq.keys():
            #get priority 
            if valFreq[key]>maxAttr:
                major=key
                maxAttr=valFreq[key]
                
        return major

    def makeTree(self,data, attributes, target, recursion):
        recursion+=1
        data=data[:]
        print('data shape {}'.format(len(data[0])))
        vals=[record[attributes.index(target)] for record in data]
    
        default=self.majority(data, attributes, target)
        if not data or len(attributes)-1 <=0:
            return default
        elif vals.count(vals[0])==len(vals):
            return vals[0]
        else:
            best=self.chooseAttr(data, attributes, target)
            tree= {best:{}}
        
            for val in self.getValues(data, attributes, best):
                examples=self.getExamples(data,attributes, best, val)
                newattr=attributes[:]
                newattr.remove(best)
                subtree=self.makeTree(examples, newattr, target, recursion)
                tree[best][val]=subtree
            
        return tree

    def predictOutput(self,testdata, attributes):
        count=0
        #print('testData {}'.format(testdata))
        res=[]
        for entry in testdata:
            #print("***************")
            #print('entry = {}'.format(entry))
            count+=1
            #print(dict_)
            tempD=dict_.copy()
            #print(isinstance(tempD, dict))
            result=""
            while isinstance(tempD, dict):
                #print('in while loop')
                #print(list(tempD.keys())[0])
                #print(tempD[list(tempD.keys())[0]].values())
                root=TreeNode.TreeNode(list(tempD.keys())[0], tempD[list(tempD.keys())[0]])
                #root=TreeNode.TreeNode(list(tempD.keys())[0], tempD[list(tempD.keys())[0]])
                #print('root = {}'.format(root))
                tempD=tempD.get(list(tempD.keys())[0])
                #print('tempD = {}'.format(tempD))
                index=attributes.index(str(root.value))
                #print('index={}'.format(index))
                value=entry[index]
                #print('value = {}'.format(value))
                #print('tempD keys = {}'.format(tempD.keys()))
                #print('row to predict {}'.format(entry))
                if value in tempD.keys():
                    child=TreeNode.TreeNode(value, tempD[value])
                    result=tempD[value]
                    tempD=tempD[value]
                else:
                    print('unable to predict')
                    result="?"
                    break
            print('testdate row {}, predicted class {}'.format(count, result))
            res.append(result)

In [ ]:
obj=DecisionTree()
training_data, testing_data, attributes, target, test_labels, data=obj.preprocessData("dt-data.txt")

In [ ]:
testing_data

In [ ]:
test_labels

In [ ]:
    dict_=obj.makeTree(training_data,attributes, target,0)

In [ ]:
dict_

In [ ]:
obj.predictOutput(testing_data,[a for a in attributes if a!='Enjoy'])

In [ ]:
obj.predictOutput(training_data,[a for a in attributes if a!='Enjoy'])

In [ ]:
testing_data

In [ ]:
training_data